## Import packages

In [19]:
import numpy as np
import pandas as pd
import gc

In [2]:
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Dropout, Reshape, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate, Concatenate
from keras.layers import Conv1D, GlobalMaxPooling1D, GlobalAveragePooling1D, Input, SpatialDropout1D, Bidirectional
from keras.layers.recurrent import LSTM, GRU
from keras import backend as K
from keras.preprocessing import sequence, text
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import optimizers, initializers, regularizers, constraints
from keras.engine.topology import Layer

Using TensorFlow backend.


In [3]:
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

## Hyper parameter setting

In [4]:
token = 'words' # based on words or chars
embed_size = 300 # how big is each word vector
max_features = 20890 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 15 # max number of words in a comment to use
num_rnn_units = 128
num_hidden_units = 300
drop_prob = 0.1
max_norm = 5.0
filter_sizes = [1,2,3,5]
num_filters = 128

## File path

In [5]:
TRAIN_PATH = './train.csv'
TEST_PATH = './test.csv'
QUESTION_PATH = './question.csv'
embed_files = {'words': './word_embed.txt', 'chars': './char_embed.txt'}

## Some helper function

In [6]:
# Get question id from a list. Remove the Q
def get_ids(qids):
    ids = []
    for t_ in qids:
        ids.append(int(t_[1:]))
    return np.asarray(ids)

# Get the text
def get_texts(q_list, question_path=QUESTION_PATH):
    qes = pd.read_csv(question_path)
    ids = get_ids(q_list)
    all_tokens = qes[token]
    texts = [all_tokens[t] for t in ids]
    return texts

## Read the text

#### Train data

In [7]:
train = pd.read_csv(TRAIN_PATH)
x_train = list(zip(train['q1'], train['q2']))
y_train = train['label']

# get the text list of question 1 and 2
q1_train = [i[0] for i in x_train]
text1_train = get_texts(q1_train)
q2_train = [i[1] for i in x_train]
text2_train = get_texts(q2_train)

#### Test data

In [8]:
test = pd.read_csv(TEST_PATH)
x_test = list(zip(test['q1'], test['q2']))

# get the text list of question 1 and 2
q1_test = [i[0] for i in x_test]
text1_test = get_texts(q1_test)
q2_test = [i[1] for i in x_test]
text2_test = get_texts(q2_test)

## Tokenize

In [9]:
tokenizer = Tokenizer(num_words=max_features, lower=False) # Don't lower the W or L!!!
tokenizer.fit_on_texts(pd.read_csv(QUESTION_PATH)[token])

# train set
tokenized1_train = tokenizer.texts_to_sequences(text1_train)
tokenized2_train = tokenizer.texts_to_sequences(text2_train)
X1_train = pad_sequences(tokenized1_train, maxlen=maxlen)
X2_train = pad_sequences(tokenized2_train, maxlen=maxlen)

# test set
tokenized1_test = tokenizer.texts_to_sequences(text1_test)
tokenized2_test = tokenizer.texts_to_sequences(text2_test)
X1_test = pad_sequences(tokenized1_test, maxlen=maxlen)
X2_test = pad_sequences(tokenized2_test, maxlen=maxlen)

## Prepare the pretrained word embedding

In [10]:
def get_coefs(line): return line[0], np.asarray(line[1:], dtype='float32')
embed_file = embed_files[token]
embeddings_index = dict(get_coefs(o.strip().split()) for o in open(embed_file, encoding='utf-8'))

20891


In [11]:
all_embs = np.hstack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()

(0.015683081, 1.1956546)

In [12]:
word_index = tokenizer.word_index
embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features+1, embed_size))

for word, i in word_index.items():
    if i > max_features: break
    embedding_vector = embeddings_index.get(word)
    #print (i, word, len(embedding_vector))
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [13]:
embedding_matrix = np.asarray(embedding_matrix, dtype='float32')

## Build the model

In [14]:
K.clear_session()

In [ ]:
# Attetion layer
class FeedForwardAttention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        """
        Keras Layer that implements an Attention mechanism for temporal data.
        Follows the work of Raffel et al. [https://arxiv.org/abs/1512.08756]
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(Attention())
        """
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(FeedForwardAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def call(self, x):

        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)), K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.softmax(eij)

        a = K.expand_dims(a)
        weighted_input = x * a

        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        #return input_shape[0], input_shape[-1]
        return input_shape[0],  self.features_dim

In [15]:
def single_model():
    inp1 = Input(shape=(maxlen,))
    inp2 = Input(shape=(maxlen,))

    # basic rnn+cnn+attention model
    inp = Input(shape=(maxlen,))
    h = Embedding(max_features+1, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    h = SpatialDropout1D(drop_prob)(h)
    h = Bidirectional(LSTM(num_rnn_units, return_sequences=True, dropout=drop_prob, recurrent_dropout=drop_prob))(h)
    
    conv_0 = Conv1D(num_filters, kernel_size=filter_sizes[0], padding = "same", activation = 'relu')(h)
    conv_1 = Conv1D(num_filters, kernel_size=filter_sizes[1], padding = "same", activation = 'relu')(h)
    conv_2 = Conv1D(num_filters, kernel_size=filter_sizes[2], padding = "same", activation = 'relu')(h)
    conv_3 = Conv1D(num_filters, kernel_size=filter_sizes[3], padding = "same", activation = 'relu')(h)

    maxpool_0 = GlobalMaxPooling1D()(conv_0)
    avgpool_0 = GlobalAveragePooling1D()(conv_0)
    att_0 = FeedForwardAttention(maxlen)(conv_0)
    maxpool_1 = GlobalMaxPooling1D()(conv_1)
    avgpool_1 = GlobalAveragePooling1D()(conv_1)
    att_1 = FeedForwardAttention(maxlen)(conv_1)
    maxpool_2 = GlobalMaxPooling1D()(conv_2)
    avgpool_2 = GlobalAveragePooling1D()(conv_2)
    att_2 = FeedForwardAttention(maxlen)(conv_2)
    maxpool_3 = GlobalMaxPooling1D()(conv_3)
    avgpool_3 = GlobalAveragePooling1D()(conv_3)
    att_3 = FeedForwardAttention(maxlen)(conv_3)

    z = concatenate([maxpool_0, maxpool_1, maxpool_2, maxpool_3, avgpool_0, avgpool_1, avgpool_2, avgpool_3, att_0, att_1, att_2, att_3])

    base_model = Model(inputs=inp, outputs=z)

    o1 = base_model(inp1)
    o2 = base_model(inp2)

    conc = concatenate([o1,o2])
    x = BatchNormalization()(conc)

    x = Dense(num_hidden_units, activation='relu')(x)
    x = Dropout(drop_prob)(x)
    x = BatchNormalization()(x)

    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=[inp1, inp2], outputs=x)
    
    return model

## Train the model and predict to make the first layer feature
Split the train set into 8 folds. Train the model using 7 folds.

Predict the remaining 1 fold and concatenate. The sequence is not changed in order to unify different models.

Predict the test set and take average. The sequence is unchanged.

In [16]:
kf = KFold(n_splits=8)
n_fold = 0
val_pred_list = []
test_pred_list = []
y_val_list = []
for train_index, valid_index in kf.split(X1_train):
    n_fold += 1
    print ("========== Fold %d: =========="%n_fold)
    
    # split samples
    X1_tra, X1_val = X1_train[train_index], X1_train[valid_index]
    X2_tra, X2_val = X2_train[train_index], X2_train[valid_index]
    y_tra, y_val = y_train[train_index], y_train[valid_index]
    
    # build the model
    #K.clear_session()
    adam = optimizers.Adam(clipnorm=max_norm)
    model = single_model()
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    # train the model
    #cp = ModelCheckpoint(filepath="my_model11-%d.h5"%n_fold, save_best_only=True)
    es = EarlyStopping(patience=4)
    rp = ReduceLROnPlateau(patience = 0)
    hist = model.fit([X1_tra, X2_tra], y_tra, batch_size = 256, epochs=20, validation_data=([X1_val, X2_val], y_val), callbacks=[es, rp])
    
    # load the best checkpoint and predict
    #K.clear_session()
    #del model
    #model = load_model("my_model11-%d.h5"%n_fold)
    val_pred = model.predict([X1_val, X2_val], batch_size=1024)
    val_pred_list.append(val_pred)
    y_val_list.append(y_val)
    test_pred = model.predict([X1_test, X2_test], batch_size=1024)
    test_pred_list.append(test_pred)
    del X1_tra, X1_val, X2_tra, X2_val, y_tra, y_val, model, val_pred, test_pred, hist
    gc.collect()

========== Fold 1: ==========
Train on 222587 samples, validate on 31799 samples
Epoch 1/1
222587/222587 [==============================] - 73s 326us/step - loss: 0.4612 - acc: 0.7812 - val_loss: 0.3457 - val_acc: 0.8465
2
3
4
5
========== Fold 2: ==========
Train on 222587 samples, validate on 31799 samples
Epoch 1/1
222587/222587 [==============================] - 71s 320us/step - loss: 0.4585 - acc: 0.7840 - val_loss: 0.3422 - val_acc: 0.8478
2
3
4
5
========== Fold 3: ==========
Train on 222588 samples, validate on 31798 samples
Epoch 1/1
222588/222588 [==============================] - 71s 320us/step - loss: 0.4622 - acc: 0.7818 - val_loss: 0.3383 - val_acc: 0.8494
2
3
4
5
========== Fold 4: ==========
Train on 222588 samples, validate on 31798 samples
Epoch 1/1
222588/222588 [==============================] - 72s 324us/step - loss: 0.4624 - acc: 0.7809 - val_loss: 0.3449 - val_acc: 0.8468
2
3
4
5
========== Fold 5: ==========
Train on 222588 samples, validate on 31798 samples
Epo

In [17]:
val_pred_list = np.concatenate(val_pred_list)
test_pred_list = np.asarray(test_pred_list)
y_val_list = np.concatenate(y_val_list)
test_pred_list = np.mean(test_pred_list, axis=0)

In [26]:
val_pred_list = np.squeeze(val_pred_list)
test_pred_list = np.squeeze(test_pred_list)

## Write the array into csv file

In [30]:
train = pd.DataFrame({'train':val_pred_list, 'label':y_val_list})
test = pd.DataFrame({'test':test_pred_list})
train.to_csv('train14.csv', index=False)
test.to_csv('test14.csv', index=False)